# Using the DWA-a102 library

**created by edwin salazar and carlos guevara**

about this notebook: shows the usage of the above mentioned library for the water balance calculation of different surfaces and and corresponding measures for the runoff uptake 

In [1]:
# import standard libraries
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# import dwa-a102 libraries
from dwa_a102 import StudyArea as StudyArea
from dwa_a102 import watbal as watbal
from check_ranges import validRange
from climate import climate

### Example # 1 

In Wuppertal the administrator of a residential building desire to install a rainwater in-situ measure to reduce the runoff. The building has a built area of 750 m$^{2}$. The desired measure is an infiltration swale (hydraulic conductivity of the soil = 20 mm h$^{-1}$). 

<br></br>
<center><img src="images/example_1.png" style="width: 300px"/></center>
<br></br>

**Calculate the required area to infiltrate the rainwater and report the main components of the water balance.**


As first step, we define the study area. This is done using <b>StudyArea</b>. Since we know the specific location, we can give it as an input parameter (i.e. check <b>climate</b> to see the locations available). If not, precipitation and evapotranspiration can be given as parameters.   

In [2]:
# create study area object
Wuppertal = StudyArea(location='Wuppertal')

Check which precipitation and evapotranspiration are assigned to Wuppertal.

In [3]:
# print precipitation and evapotranspiration for study area
print(Wuppertal)

Study area has a precipitation of 1400 mm/a, and potential evapotranspiration of 525 mm/a


A flat area with 750 m$^{2}$ is assumed for the residential building. Use the <b>flat_roof</b> method to calculate the corresponding coefficients. 

In [4]:
# calculate a flat area
roof = Wuppertal.roof(750)

The flat area give the following results:

In [5]:
display(roof)

,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Roof,750,714.0,1400,525,0.952,0,0.048,0,1050,1000.0,0,50.0,0


In order to capture the runoff produced by the flat area, the measure selected is an infiltration swale. Use the <b>infilt_swale</b> method, which uses the hydraulic conductivity of the soil (20 mm h −1) and the corresponding surface(s) as input. 

In [6]:
infilSwale = Wuppertal.infilt_swale(20, roof)

The infiltation swale gives the following results:

In [7]:
display(infilSwale)

,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Roof,750,714.0,1400,525,0.952,0,0.048,0,1050,0.0,0,50.0,0
1,Infilt. swale,118,714.0,1400,525,0.009,0.959,0.032,0,165,11.0,1117,37.0,0


The corresponding water balance is calculated using the <b>watbal</b> function, which uses the surfaces and measures taken as input. The water balance is as follows:

In [8]:
display(watbal(roof,infilSwale))

,Element,Area,a,g,v,Vp,Va,Vg,Vv
0,Roof,750,0.952,0,0.048,1050,1000.0,0,50.0
0,Roof,750,0.952,0,0.048,1050,0.0,0,50.0
1,Infilt. swale,118,0.009,0.959,0.032,165,11.0,1117,37.0
0,System,1618,0.446,0.493,0.060,2265,1011.0,1117,137.0


### Example # 2 

A neighborhood in München requires an analysis of the water urban cycle. All buildings have a built area of 750 m$^{2}$. Calculate the required area to infiltrate the rainwater of Building 1 (hydraulic conductivity of the soil = 150 mm h$^{-1}$). Building 2 and Building 3 drain with pipes the rainwater. Additionally,  both building share a public area of 1500 m$^{2}$ and its ground is mode of partially permeable (FA = 5%, kf = 50 mm h$^{-1}$, Sp = 1 mm). There is a street completely impermeable that divide Building 1 from B. 2 and B. 3., storage height of the street is 2.5 mm. 

<br></br>
<center><img src="images/example_2.png" style="width: 300px"/></center>
<br></br>

**Calculate the required area to infiltrate the rainwater and report the main components of the water balance.**

In [9]:
# create study area object
Munich = StudyArea(location='München')

In [10]:
# print precipitation and evapotranspiration for study area
print(Munich)

Study area has a precipitation of 1000 mm/a, and potential evapotranspiration of 550 mm/a


Calculate the runoff produced by the three buildings.

In [11]:
# create flat areas for all buildings

# initialize flat areas list
roofs = []

# store the flat area for each building
for i in range(3):
    roofs.append(Munich.roof(750))

In [12]:
# Building 1 - flat area
display(roofs[0])


,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Roof,750,693.0,1000,550,0.924,0,0.076,0,750,693.0,0,57.0,0


Use and infiltation swale to capture the runoff from Building 1

In [13]:
infilSwale_B1 = Munich.infilt_swale(150, roofs[0])

The information for infilSwale_B1 is as follows:

In [14]:
display(infilSwale_B1)

,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Roof,750,693.0,1000,550,0.924,0,0.076,0,750,0.0,0,57.0,0
1,Infilt. swale,61,693.0,1000,550,0.000,0.974,0.026,0,61,0.0,734,20.0,0


Building 2 and 3 use a pipe as drainage measure. Calculate this measure using the <b>drainage</b> method. 

In [15]:
# Drainage for building 2
drainage_B2 = Munich.drainage("pipe", roofs[1])
# Drainage for building 3
drainage_B3 = Munich.drainage("pipe", roofs[1])

In [16]:
display(drainage_B2)

,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Roof,750,693.0,1000,550,0.924,0,0.076,0,750,0.0,0,57.0,0
1,Drainage,0,693.0,1000,550,1.000,0,0.000,0,0,693.0,0,0.0,0


Moreover, building 2 and 3 share a common infiltration area with an area of 1500 m$^{2}$. Create the infiltation area as specified (use <b>permeable_surface</b>) above.  

In [17]:
green_area_B2_3 = Munich.permeable_surface(1500, 5, 50, sp=1)

In [18]:
display(green_area_B2_3)

,Element,Area,Au,P,Etp,a,g,v,e,Vp,Va,Vg,Vv,Ve
0,Permeable surface,1500,441.0,1000,550,0.294,0.566,0.14,0,1500,441.0,848.0,210.0,0


The street runoff is calculated using the <b>flat_area</b> method. The area of the street is assumed to be 1000 m$^{2}$ with a storage height of 2.5 mm. 

In [19]:
street = Munich.flat_area(1000, 2.5)

The water balance for the Munich area is as follows:

In [20]:
display(watbal(
    roofs[0], 
    roofs[1], 
    roofs[2], 
    infilSwale_B1, 
    drainage_B2, 
    drainage_B2,
    green_area_B2_3, 
    street
))

,Element,Area,a,g,v,Vp,Va,Vg,Vv
0,Roof,750,0.924,0,0.076,750,693.0,0,57.0
0,Roof,750,0.924,0,0.076,750,693.0,0,57.0
0,Roof,750,0.924,0,0.076,750,693.0,0,57.0
0,Roof,750,0.924,0,0.076,750,0.0,0,57.0
1,Infilt. swale,61,0.000,0.974,0.026,61,0.0,734,20.0
0,Roof,750,0.924,0,0.076,750,0.0,0,57.0
1,Drainage,0,1.000,0,0.000,0,693.0,0,0.0
0,Roof,750,0.924,0,0.076,750,0.0,0,57.0
1,Drainage,0,1.000,0,0.000,0,693.0,0,0.0
0,Permeable surface,1500,0.294,0.566,0.140,1500,441.0,848,210.0
